In [1]:
# =================================================================
# 1. CONFIGURACIÓN DEL ENTORNO
# =================================================================
import os
import warnings
import langchain
from dotenv import load_dotenv

load_dotenv()

# Parche de compatibilidad
if not hasattr(langchain, 'verbose'):
    langchain.verbose = False

from langchain_groq import ChatGroq
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

# Solución para errores de definición de Pydantic
try:
    from langchain_core.caches import BaseCache
    from langchain_core.callbacks import Callbacks
    ConversationSummaryBufferMemory.model_rebuild()
except Exception:
    pass

warnings.filterwarnings('ignore')

# =================================================================
# 2. INICIALIZACIÓN DEL MODELO Y MEMORIA
# =================================================================

# Usamos ChatGroq directamente para evitar errores de conteo de tokens
llm = ChatGroq(
    temperature=0.0,
    model_name="llama-3.1-8b-instant",
    groq_api_key=os.environ.get("GROQ_API_KEY")
)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True 
)

# =================================================================
# 3. PRUEBA DE LA CONVERSACIÓN
# =================================================================

print("--- Turno 1 ---")
print(conversation.predict(input="Hola, soy Luciano, Analista Programador de Argentina."))

print("\n--- Turno 2 (Llenando buffer de tokens) ---")
texto_largo = """
Mi agenda para mañana:
9:00 - Review de código con el equipo senior.
11:00 - Entrevista técnica en Ualá.
14:00 - Despliegue a producción de la nueva API.
16:00 - Gimnasio.
"""
print(conversation.predict(input=f"Anotá mi agenda: {texto_largo}"))

print("\n--- Turno 3 (Verificando memoria histórica) ---")
print(conversation.predict(input="¿Qué recordás de mi perfil y qué planes tengo para mañana?"))

--- Turno 1 ---


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, soy Luciano, Analista Programador de Argentina.
AI:

> Finished chain.
Hola Luciano, Analista Programador de Argentina. Me alegra conocerte. ¿De qué parte de Argentina eres? ¿Te gusta trabajar con programación?

--- Turno 2 (Llenando buffer de tokens) ---


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, soy Luciano, Analista Programador de Argentina.
AI: Hol

¿Cómo funcionó en tu código?
Fijate en lo que salió en tu consola en el Turno 3:

Superaste el límite: Como pusimos max_token_limit=100 (un límite muy bajo), en cuanto le pasaste la agenda larga, el programa dijo: "Upa, si guardo todo esto tal cual, me paso de 100 tokens".
El Resumen automático: Antes de responder, LangChain llamó a la IA por detrás y le pidió: "Resumime todo lo que hablamos hasta ahora".
El nuevo Prompt: Si miras el Turno 3, verás que dice: System: Current summary: Luciano, an Argentine programmer analyst... La IA ya no está leyendo "Hola soy Luciano...", está leyendo el resumen condensado.
¿Por qué esto es mejor que otras memorias?
Si usás ConversationBufferMemory (la básica): Tarde o temprano te quedás sin memoria o la cuenta de la API se va a las nubes porque el historial crece infinitamente.
Si usás ConversationSummaryBufferMemory: Nunca te pasás del límite. La conversación puede durar 10 horas, que la IA siempre va a tener un resumen de lo importante y las últimas 2 o 3 frases exactas para mantener el contexto fluido.